# Setup Data

## Import libraries

In [2]:
import boto3
import os
import pandas as pd
import sagemaker

from IPython.core.display import display, HTML
from pyathena import connect

/tmp/ipykernel_2009/4216669510.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# check dependencies are stored
%store

Stored variables and their in-db values:
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


## Setup S3 Bucket

In [5]:
# save Amazon information
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
session = sagemaker.Session()

#bucket = session.default_bucket()
#sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [7]:
print(region)

us-east-1


In [9]:
# create a new S3 bucket
s3 = boto3.client('s3')
bucket_name = 'ai-cooking-assistant-5432'
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '18WR8N2J6SX1EBA9',
  'HostId': 'isuk4guujxV5SgVZQNjGleaN8Rdi4F2FuFiqdsSTg28+VINMoTCZnVbl6+KzVDFBjYJvMz4sul4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'isuk4guujxV5SgVZQNjGleaN8Rdi4F2FuFiqdsSTg28+VINMoTCZnVbl6+KzVDFBjYJvMz4sul4=',
   'x-amz-request-id': '18WR8N2J6SX1EBA9',
   'date': 'Thu, 23 May 2024 01:36:16 GMT',
   'location': '/ai-cooking-assistant-5432',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/ai-cooking-assistant-5432'}

## Copy Data to our S3 Bucket

In [14]:
# save path to wizards of tasks bucket
s3_public_path = "s3://wizard-of-tasks/"
print(s3_public_path)

s3://wizard-of-tasks/


In [15]:
# save path to S3 bucket
s3_path = "s3://{}/data/".format(bucket_name)
print(s3_path)

s3://ai-cooking-assistant-5432/data


In [19]:
# copy data from local to s3
!aws s3 cp $s3_public_path $s3_path

In [18]:
# check it uploaded
display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

In [24]:
# List objects in the source bucket
source_bucket = "wizard-of-tasks"
response = s3.list_objects_v2(Bucket=source_bucket,)

ParamValidationError: Parameter validation failed:
Invalid bucket name "arn:aws:s3:::wizard-of-tasks": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:s3:[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"

In [27]:
source_bucket = "wizard-of-tasks"
# Create Boto3 client for S3 with the specified region
s3 = boto3.client('s3', region_name='us-west-2')

# List objects in the source bucket
response = s3.list_objects_v2(Bucket=source_bucket)

print(response)

{'ResponseMetadata': {'RequestId': 'SN0FK2DT1S844DTP', 'HostId': 'L/8HghjsSC+5mn+CWayj/QJVueVd3p8v81ZVpvsRGuj4ghvosmhmIVbExIcG+w/drERP7bispHkLqeRdXzr1iA==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'L/8HghjsSC+5mn+CWayj/QJVueVd3p8v81ZVpvsRGuj4ghvosmhmIVbExIcG+w/drERP7bispHkLqeRdXzr1iA==', 'x-amz-request-id': 'SN0FK2DT1S844DTP', 'date': 'Thu, 23 May 2024 02:03:54 GMT', 'x-amz-bucket-region': 'us-west-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'README.md', 'LastModified': datetime.datetime(2022, 10, 7, 22, 59, 54, tzinfo=tzlocal()), 'ETag': '"16a8aa13b9d476be1ddf41491896d3b7"', 'Size': 6354, 'StorageClass': 'STANDARD'}, {'Key': 'wizard_of_tasks_cooking_v1.0.json', 'LastModified': datetime.datetime(2022, 10, 7, 23, 1, 45, tzinfo=tzlocal()), 'ETag': '"09619ab31682ca2235992f26fd986235"', 'Size': 3826844, 'StorageClass': 'INTELLIGENT_TIERING'}, {'Key': 'wizard_of_tas

## Shut down notebook resources

In [26]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [27]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>